输出模型整体结构

In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
model_path = r"E:\gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)
model.eval()
# model = model.half()  
wte=1
wpe=1
# print(model)
with torch.no_grad():
    for name, param in model.named_parameters():
        print(f"Layer: {name} | Shape Type: {param.shape}")
        # if name == "transformer.ln_f.weight":
        #     print(f"Layer: {name} | Shape Type: {param.shape}| Data Type: {param}")
        # if name == "transformer.ln_f.bias":
        #     print(f"Layer: {name} | Shape Type: {param.shape} | Data Type: {param}")
#             df = pd.DataFrame(param.numpy())
#             # df.to_csv('output.csv', index=False)  
#         # print(f"Layer: {name} | Shape: {param.shape}")
#     # for name, module in model.named_modules():
#     #      print(f"Name: {name}, Module: {module}")

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM,LlamaModel
model_path = r"D:\ProjectRust\learning-lm-rs\models\story"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)
model.eval()
# model = model.half()  
print(model)

In [ ]:
# 可视化模型的计算流图
from transformers import AutoTokenizer, GPT2LMHeadModel 
import torch
# 定义一个钩子函数，用于打印每一层的输入和输出
model_path = r"E:\gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path).eval()
# 遍历所有子模块并注册钩子

# 手动构建张量
inputs = torch.tensor([])
print(tokenizer.decode([717,198,717]))

# model.eval()
# model.generate(inputs, max_length=50)


In [ ]:
from transformers import AutoTokenizer, GPT2LMHeadModel
import torch

# 加载模型和分词器
model_path = r"E:\gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path).eval()

def find_layernorm_eps(module, prefix=""):
    """递归地查找所有的 LayerNorm 层并打印其 epsilon 值"""
    for name, child in module.named_children():
        full_name = f"{prefix}.{name}" if prefix else name
        if isinstance(child, torch.nn.LayerNorm):
            print(f"LayerNorm found at {full_name} with epsilon: {child.eps}")
        find_layernorm_eps(child, full_name)

# 打印所有 LayerNorm 层的 epsilon 值
find_layernorm_eps(model)

# 如果你想要获取特定 LayerNorm 层的 epsilon 值，可以直接访问该层
# 例如，假设我们知道某个 LayerNorm 层的位置如下：
# layer_norm = model.transformer.h[0].ln_1
# print(layer_norm.eps)

构造回调函数，用于输出每一层的计算结果。

In [ ]:
from transformers import AutoTokenizer, GPT2LMHeadModel 
import torch
def print_tensor_elements(tensor, label="Tensor", num_elements=5):
    if isinstance(tensor, torch.Tensor):
        elements = tensor.flatten()
        print(f"{label}: shape={tensor.shape}")
        print("First 5 elements:", elements[:num_elements].tolist())
        print("Last 5 elements:", elements[-num_elements:].tolist() if len(elements) >= num_elements else elements.tolist())
    else:
        print(f"{label}: Not a Tensor")

def hook_fn(module, input, output):
    print(f"Layer: {module.__class__.__name__}")

    # 处理输入张量
    if input and isinstance(input[0], torch.Tensor):
        print_tensor_elements(input[0], label="Input")
    else:
        print("error")
    # 处理输出张量
    if input and isinstance(output[0], torch.Tensor):
        print_tensor_elements(output[0], label="Input")
    else:
        print("error")


    print("-" * 50)

# 加载预训练模型和分词器
model_path = r"E:\gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path).eval()
# 遍历所有子模块并注册钩子
hooks = []
for name, module in model.named_modules():
    if not isinstance(module, (torch.nn.ModuleList, torch.nn.Sequential)):
        hooks.append(module.register_forward_hook(hook_fn))
        # if name == 'transformer.ln_f':
        #     hooks.append(module.register_forward_hook(hook_fn))

# 手动构建张量并进行推理
inputs = "Once upon a time,"
generated_tokens = torch.tensor([[0]])
# 将文本转换为模型输入
input_ids = tokenizer(inputs, return_tensors='pt').input_ids

# 使用模型进行推理
with torch.no_grad():  # 确保推理过程中不计算梯度以节省内存
    outputs = model.generate(generated_tokens, max_length=10, do_sample=True)
print(outputs)
for i in range(outputs.shape[0]):  # 遍历所有生成的序列
    print(tokenizer.decode(outputs[i], skip_special_tokens=True))

In [ ]:
from transformers import AutoTokenizer, GPT2LMHeadModel
import torch

# 加载预训练模型和分词器
model_path = r"E:\gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path).eval()

# 已经生成的token列表
generated_tokens = [4874, 2402, 257, 640, 11]

# 将token列表转换为tensor
input_ids = torch.tensor([generated_tokens])

# 设置生成参数
generation_kwargs = {
    'max_length': len(generated_tokens) + 30,  # 基于已有的tokens再生成30个
    'min_length': len(generated_tokens) + 30,  # 确保至少生成30个新词
    'num_beams': 5,                             # 使用波束搜索，值越大生成质量越高但速度越慢
    'no_repeat_ngram_size': 2,                  # 避免重复的n-gram
    'early_stopping': True                      # 如果达到良好的结果就提前停止生成
}

# 使用模型进行推理并生成指定数量的词
with torch.no_grad():  # 推理时不需要计算梯度
    output = model.generate(input_ids, **generation_kwargs)

# 解码模型生成的输出
decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)

print("模型生成的文本:", decoded_output)

#

In [ ]:
from transformers import AutoTokenizer, GPT2LMHeadModel
import torch

# 加载预训练模型和分词器
model_path = r"E:\gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path).eval()

for name, layer in model.named_modules():
    print(name)


In [ ]:
# cuda test
import torch

# 检查是否有可用的CUDA设备
print(f"Is CUDA available: {torch.cuda.is_available()}")

# 打印PyTorch版本
print(f"PyTorch version: {torch.__version__}")

# 获取当前默认的CUDA设备
if torch.cuda.is_available():
    current_device = torch.cuda.current_device()
    print(f"Current CUDA device: {current_device}")
    print(f"Name of the current CUDA device: {torch.cuda.get_device_name(current_device)}")
else:
    print("No CUDA devices available.")

# 列出所有可用的CUDA设备数量
print(f"Number of available CUDA devices: {torch.cuda.device_count()}")

In [ ]:
from transformers import AutoTokenizer, GPT2LMHeadModel 
import torch
# 加载预训练模型和分词器
model_path = r"E:\gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path).eval()

def generate_text_word_by_word(model, tokenizer, start_text, max_length=50):
    # 将起始文本编码为模型输入
    input_ids = tokenizer.encode(start_text, return_tensors='pt')

    generated = []
    for _ in range(max_length):
        # 模型推理
        with torch.no_grad():
            outputs = model(input_ids)
            next_token_logits = outputs.logits[:, -1, :]

            # 对最后一个token应用softmax获取概率分布
            next_token_probs = torch.nn.functional.softmax(next_token_logits, dim=-1)

            # 选择具有最高概率的下一个token
            next_token_id = torch.argmax(next_token_probs, dim=-1)

            # 如果next_token是EOS（结束符号），则停止生成
            if next_token_id.item() == tokenizer.eos_token_id:
                break

            # 添加到生成的列表中，并更新input_ids以便下一轮迭代使用
            generated.append(next_token_id.item())
            input_ids = torch.cat([input_ids, next_token_id.unsqueeze(0)], dim=-1)

    # 解码生成的token id为文本
    return tokenizer.decode(generated, skip_special_tokens=True)

# 示例文本
start_text = "Once upon a time,"

# 逐词生成文本
generated_text = generate_text_word_by_word(model, tokenizer, start_text, max_length=90)
print("Generated text:", generated_text)

In [ ]:
# 转换半精度
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
model_path = r"E:\gpt2"
model_path1 = r"E:\gpt2t"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)
model = model.half()
model.save_pretrained(model_path1)
tokenizer.save_pretrained(model_path1)